In [28]:
from bs4 import BeautifulSoup
import requests
import csv
import os

In [71]:
positions = {
    'QB': 0,
    'RB': 2,
    'WR': 4,
    'TE': 6,
    'DST': 16,
    'K': 17,
    'FLEX': 23
}
weeks = range(1, 22)

In [72]:
def cbs(pos, week):
    url = 'http://games.espn.com/ffl/tools/projections?&scoringPeriodId={week}&seasonId=2017&slotCategoryId={pos}'.format(pos=positions[pos], week=week)
    try:
        contents = requests.get(url).content
        soup = BeautifulSoup(contents, "html.parser")
        rows = soup.find('table').find_all('tr')
        header = [td.text.split(',')[0].replace(u'\xa0', u'') for td in rows[2]]
        data = [[td.text.split(',')[0].split('D/ST')[0] for td in row] for row in rows[3:]]
    except AttributeError:
        print("Failed: " + pos + " " + str(week))
        return
    filename = 'espn_{pos}_week{week}.csv'.format(pos=pos, week=week)
    with open(filename, 'w') as f:
        print('writing: ', filename)
        writer = csv.writer(f)
        try:
            writer.writerow(header)
            writer.writerows(data)
        except UnicodeEncodeError:
            print("Failed: " + pos + " " + str(week))
            print(header)
            print(data)
            return

In [73]:
os.chdir('espn')
[cbs(pos, week) for pos in positions for week in weeks]
os.chdir('..')

writing:  espn_QB_week1.csv
writing:  espn_QB_week2.csv
writing:  espn_QB_week3.csv
writing:  espn_QB_week4.csv
writing:  espn_QB_week5.csv
writing:  espn_QB_week6.csv
writing:  espn_QB_week7.csv
writing:  espn_QB_week8.csv
writing:  espn_QB_week9.csv
writing:  espn_QB_week10.csv
writing:  espn_QB_week11.csv
writing:  espn_QB_week12.csv
writing:  espn_QB_week13.csv
writing:  espn_QB_week14.csv
writing:  espn_QB_week15.csv
writing:  espn_QB_week16.csv
writing:  espn_QB_week17.csv
writing:  espn_QB_week18.csv
Failed: QB 19
Failed: QB 20
Failed: QB 21
writing:  espn_RB_week1.csv
writing:  espn_RB_week2.csv
writing:  espn_RB_week3.csv
writing:  espn_RB_week4.csv
writing:  espn_RB_week5.csv
writing:  espn_RB_week6.csv
writing:  espn_RB_week7.csv
writing:  espn_RB_week8.csv
writing:  espn_RB_week9.csv
writing:  espn_RB_week10.csv
writing:  espn_RB_week11.csv
writing:  espn_RB_week12.csv
writing:  espn_RB_week13.csv
writing:  espn_RB_week14.csv
writing:  espn_RB_week15.csv
writing:  espn_RB_w

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,